# 3 Qubit sign flip code
We have seen in the previous posts that we can mitigate a storage error but only when the error is an X gate. So what happens when we have an x or a z gate applied due to the noise error?. If we try the 3-qubit code we won't get a good result. That is why it is needed to have another form of correcting this tipe of errors. To address this problematic researchers came up with the idea of the sign flip code.

We'll present you an execution of the sign flip code in the same way we presented the 3-qubit code. That is codificating strings.

In this code we make the assumption that the error happens in one part of the circuit. And also, the error only affects one qubit at a time.

First, let's define some function that will be useful for later.  The first two functions transform a string to bits and vice versa. The get_noise applies a x gate on a random qubit if a probability check up is met and finally codificate which codificate a bit string into its equivalent in qubits. We simulate the error in this way to have a controled way of testing and verifying if our code works.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute, ClassicalRegister
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import lookuptable_decoding
from qiskit.ignis.verification.topological_codes import GraphDecoder
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)
from qiskit.visualization import plot_histogram
import random
backend = Aer.get_backend('qasm_simulator')

def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def get_noise(circuit,probability,qubits):
    random_number = random.uniform(0, 1)
    if(random_number <= probability):
        qubit = random.randint(0,len(qubits)-1)
        circuit.z(qubit)
    return circuit

def codificate(bitString):
    qubits = list()
    for i in range(len(bitString)):
        mycircuit = QuantumCircuit(1,1)
        if(bitString[i] == "1"):
            mycircuit.x(0)
        mycircuit.h(0)
        qubits.append(mycircuit)
    return qubits

Let's see the result for the string without any noise on the circuit. The original string is I like dogs, we hope to get the same string once we implemented the error correction code.

In [23]:
m0 = tobits("I like dogs") 
qubits = codificate(m0)
measurements = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.h(0)
    qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))
print(frombits(measurements))

I like dogs


As you can see, the string without any noise is just "I like dogs" like the original string.

Next, if we add some noise then the result will be the following

In [32]:
m0 = tobits("I like dogs") 
qubits = codificate(m0)
measurements = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit = get_noise(qubit,0.2,range(qubit.num_qubits))
    qubit.h(0)
    qubit.measure(0,0)    
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))
print(frombits(measurements))

I*È{jm â'{


As you can see introduction of error, even if it's with a small probability, could yield to very bad results.
The next thing we'll do is fix the errors with the sign flip code.

The circuit for one qubit is the following:

In [4]:
cb = QuantumRegister(1,'code_qubit')
lq = QuantumRegister(4,'ancilla_qubit')
sb = ClassicalRegister(2,'syndrome_bit')
out = ClassicalRegister(1,'output_bit')
mycircuit = QuantumCircuit(cb,lq,sb,out)
mycircuit.cx(0,1)
mycircuit.cx(1,2)
mycircuit.h([0,1,2])
mycircuit.barrier()
mycircuit = get_noise(mycircuit,0.2,range(3))
mycircuit.barrier()
mycircuit.h([0,1,2])
mycircuit.cx(0,3)
mycircuit.cx(1,3)
mycircuit.cx(0,4)
mycircuit.cx(2,4)
mycircuit.measure(3,0)
mycircuit.measure(4,1)
mycircuit.draw()

┌───┐      ░       ░ ┌───┐                          
   code_qubit_0: ──■──┤ H ├──────░───────░─┤ H ├──■─────────■─────────────
                 ┌─┴─┐└───┘┌───┐ ░       ░ ├───┤  │         │             
ancilla_qubit_0: ┤ X ├──■──┤ H ├─░───────░─┤ H ├──┼────■────┼─────────────
                 └───┘┌─┴─┐├───┤ ░ ┌───┐ ░ ├───┤  │    │    │             
ancilla_qubit_1: ─────┤ X ├┤ H ├─░─┤ Z ├─░─┤ H ├──┼────┼────┼────■────────
                      └───┘└───┘ ░ └───┘ ░ └───┘┌─┴─┐┌─┴─┐  │    │  ┌─┐   
ancilla_qubit_2: ────────────────░───────░──────┤ X ├┤ X ├──┼────┼──┤M├───
                                 ░       ░      └───┘└───┘┌─┴─┐┌─┴─┐└╥┘┌─┐
ancilla_qubit_3: ────────────────░───────░────────────────┤ X ├┤ X ├─╫─┤M├
                                 ░       ░                └───┘└───┘ ║ └╥┘
 syndrome_bit: 2/════════════════════════════════════════════════════╩══╩═
                                                                     0  1 
                                                                          
   output_bit: 1/═════════════════════════════════════════════════════════

As you can see in the example, we have an error (it is the z gate). That gate was applied at random with a probability of 0.2. After we meassure the syndrome bits we'll know if an error ocurred during the proccess. To detect this error we have to apply a z gate if the values are 11 because the only possible way of meassuring 11 it was if an error occured. The possible outcomes are summariced in this table:

| Meassurement | Error location | Action |
|--------------|----------------|--------|
| 00           | No error       | No     |
| 01           | Ancilla bit    | No     |
| 10           | Ancilla bit    | No     |
| 11           | Code qubit     | Z gate |

And to address this issue (if it exists) we just have to check for this values and apply a z gate to revert the error.

Continuing with the proccess for the string "I like dogs", first we have to to apply the error correction code.

In [33]:
for i in range(len(qubits)):
    cb = QuantumRegister(1,'code_qubit')
    lq = QuantumRegister(4,'ancilla_qubit')
    sb = ClassicalRegister(2,'syndrome_bit')
    out = ClassicalRegister(1,'output_bit')
    mycircuit = QuantumCircuit(cb,lq,sb,out)
    if(m0[i] == "1"):
        mycircuit.x(0)
    qubit.h(0)
    mycircuit.cx(0,1)
    mycircuit.cx(1,2)
    mycircuit.h([0,1,2])
    mycircuit.barrier()
    mycircuit = get_noise(mycircuit,0.2,range(3))
    mycircuit.barrier()
    mycircuit.h([0,1,2])
    mycircuit.cx(0,3)
    mycircuit.cx(1,3)
    mycircuit.cx(0,4)
    mycircuit.cx(2,4)
    mycircuit.measure(3,0)
    mycircuit.measure(4,1)
    qubits[i] = mycircuit

In the code below is when we check for potential errors and when we correct them. We just check for the previous cases of the table to check for errors.

In [35]:
measurements = list()
raw_bits = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.measure(0,2)
    result = execute(qubit, backend, shots=1, memory=True).result()
    bits = result.get_memory()[0]
    raw_bits.append(int(bits[0]))
for i in range(len(qubits)):
    qubit = qubits[i]
    result = execute(qubit, backend, shots=1, memory=True).result()
    bits = result.get_memory()[0]
    if(bits[2] == '1' and bits[3] == '0'):
        qubit.x(2)
    if(bits[2] == '0' and bits[3] == '1'):
        qubit.x(1)
    if(bits[2] == '1' and bits[3] == '1'):
        qubit.x(0)
    qubit.measure(0,2)
    result = execute(qubit, backend, shots=1, memory=True).result()
    bits = result.get_memory()[0]
    measurements.append(int(bits[0]))
print("without error correction the string was: " + frombits(raw_bits))
print("with error correction the string was: " + frombits(measurements))

without error correction the string was: I mik= tg's
with error correction the string was: I like dogs


As you can see, the result is quite different with error correction and without error correction. However, what would happend if we first have an x gate and then a z gate applied to the qubits due to noise error?. Then this algorithm doesn't work anymore and we have to find another way, it results that the code is called shors code due to it's author.

## References:
Devitt, Simon & Munro, William & Nemoto, Kae. (2013). Quantum Error Correction for Beginners. Reports on progress in physics. Physical Society (Great Britain). 76. 076001. 10.1088/0034-4885/76/7/076001. 

https://en.wikipedia.org/wiki/Quantum_error_correction#The_sign_flip_code